## 1. meta data 다운로드 ##
STimage-1K4M data의 meta data를 huggingface_hub로 다운로드 합니다. 

In [1]:
from huggingface_hub import hf_hub_download

local_path = hf_hub_download(
    repo_id="jiawennnn/STimage-1K4M",      # repo ID
    filename="meta/meta_all_gene02122025.csv",  # repo 내 path
    repo_type="dataset",
    local_dir="./stimage",            # 저장할 위치
    local_dir_use_symlinks=False,
)

print(local_path)


/opt/homebrew/anaconda3/envs/hest/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/hest/lib/python3.9/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


stimage/meta/meta_all_gene02122025.csv


## 2. meta data 처리
다운로드한 meta data를 확인하고 필요한 열(slide, species, tissue, pmid, involve_cancer, tech)만 남겨 필요한 데이터의 slide 명에 쉽게 접근할 수 있도록 정리합니다. 

In [2]:
import pandas as pd

# meta data 확인
meta_raw = pd.read_csv("stimage/meta/meta_all_gene02122025.csv")
meta_raw

,slide,species,tissue,pmid,title,abstract,keywords,involve_cancer,tech,spot_num,gene_num
0,GSE144239_GSM4284316,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,666,17138
1,GSE144239_GSM4284317,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,646,17344
2,GSE144239_GSM4284318,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,638,17883
3,GSE144239_GSM4284319,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,590,16959
4,GSE144239_GSM4284320,human,skin,"32579974,38037084",Title 1: Multimodal Analysis of Composition an...,Abstract 1: To define the cellular composition...,Keywords 1: CRISPR screen; MIBI; intra-tumoral...,True,ST,521,17689
...,...,...,...,...,...,...,...,...,...,...,...
1144,Mouse_OlfactoryBulb_10X_03242022_Visium,mouse,olfactory bulb,NaN,NaN,NaN,NaN,False,Visium,1185,32285
1145,Human_Colon_10X_03252024_VisiumHD,human,colon,NaN,NaN,NaN,NaN,False,VisiumHD,545913,18085
1146,Human_Lung_10X_03292024_VisiumHD,human,lung,NaN,NaN,NaN,NaN,False,VisiumHD,605471,18085
1147,Mouse_Brain_10X_03292024_VIsiumHD,mouse,brain,NaN,NaN,NaN,NaN,False,VisiumHD,393543,19059


In [3]:
# meta data 정리
meta = meta_raw[meta_raw["species"]=="human"][["slide", "tissue", "pmid", "involve_cancer", "tech"]]
meta

,slide,tissue,pmid,involve_cancer,tech
0,GSE144239_GSM4284316,skin,"32579974,38037084",True,ST
1,GSE144239_GSM4284317,skin,"32579974,38037084",True,ST
2,GSE144239_GSM4284318,skin,"32579974,38037084",True,ST
3,GSE144239_GSM4284319,skin,"32579974,38037084",True,ST
4,GSE144239_GSM4284320,skin,"32579974,38037084",True,ST
...,...,...,...,...,...
1114,Human_Prostate_Erickson_08102022_Visium_Patien...,prostate,35948708,True,Visium
1115,Human_Prostate_Erickson_08102022_Visium_Patien...,prostate,35948708,True,Visium
1116,Human_Prostate_Erickson_08102022_Visium_Patien...,prostate,35948708,True,Visium
1145,Human_Colon_10X_03252024_VisiumHD,colon,NaN,False,VisiumHD


In [4]:
# tissue - cancer/normal 분포 확인
pd.crosstab(meta["tissue"], meta["involve_cancer"])

involve_cancer,False,True
tissue,,
brain,68,22
breast,6,189
cerebellum,1,0
cervix,1,0
colon,7,4
colorectal,1,0
endometrium,0,1
glioblastoma,13,0
glioma,6,11


-> False 혹은 True 값이 0인 경우, 슬라이드 수가 너무 적은 경우, False-True 값이 너무 많이 차이나는 경우 데이터셋 학습에 부적합합니다. HEST-1K의 데이터셋 분포도 함께 고려하여 학습에 사용할 tissue를 결정합니다. 

## 3. data 다운로드 ##
meta data에서 분포를 확인한 것을 기반으로 실제 데이터를 다운로드 합니다. 기본적으로 디렉토리 구조는 STimage-1K4M 의 구조를 따릅니다. 데이터셋은 huggingface_hub을 이용해 전체 혹은 일부만 다운로드 가능합니다. 두 경우 모두의 코드를 구현하였으며 데이터 용량 이슈로 인해 시연에서는 일부 다운로드 코드를 이용해 예시를 보입니다. 

In [ ]:
# 전체 데이터셋 다운로드

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="jiawennnn/STimage-1K4M",
    repo_type="dataset",
    local_dir="./stimage",
    local_dir_use_symlinks=False,
    resume_download=True,
)

print("다운로드 완료")

In [5]:
# 일부 데이터셋 다운로드

from huggingface_hub import snapshot_download

# 확인용, 실제 이용시 slide와 tech는 meta의 티슈를 기준으로 정리해 for 문으로 아래 과정 반봅
slide = meta.loc[0, "slide"]    # slide 이름
tech = meta.loc[0, "tech"]      # tech 종류

snapshot_download(
    repo_id="jiawennnn/STimage-1K4M",
    repo_type="dataset",
    local_dir="./stimage",          # 로컬 경로 설정
    local_dir_use_symlinks=False,
    allow_patterns=[
        f"{tech}/coord/{slide}_coord.csv",    # slide 내의 spot 위치 정보
        f"{tech}/gene_exp/{slide}_count.csv", # spot 별 유전자 발현량 raw data
        f"{tech}/image/{slide}.png"           # slide H&E image
    ],
)

/opt/homebrew/anaconda3/envs/hest/lib/python3.9/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 3 files: 100%|██████████| 3/3 [00:01<00:00,  1.51it/s]


'/Users/eunseo/Documents/2025/학교/2학기/캡스톤디자인과창업프로젝트A/st/stimage'

## 4. HEST-1K 기준 데이터 전처리
HEST-1K를 기준으로 데이터를 통합하기 위해 STimage-1K4M를 전처리하는 과정이다. 제공되는 Reader class는 이용이 불가능하므로 파이프라인을 직접 구축해 사용하였다. 

In [6]:
import scanpy as sc
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from hest import STHESTData

coord_path = Path(f"./stimage/{tech}/coord/{slide}_coord.csv")
count_path = Path(f"./stimage/{tech}/gene_exp/{slide}_count.csv")
img_path = Path(f"./stimage/{tech}/image/{slide}.png")

# 1. coord 파일 전처리

coord = pd.read_csv(coord_path, index_col=0)
coord = coord.rename(columns={'yaxis': 'Y', 'xaxis': 'X'}) # HEST-1K 요구 맞춰 수정
spot_diameter = coord.iloc[0]["r"]*2 # coord에 반지름 존재

# 바코드 형식으로 인덱스 수정
new_idx_crd = []
for idx in coord.index:
    parts = idx.rsplit('_', 1)
    if len(parts) == 2 and 'x' in parts[1]:
        row, col = parts[1].split('x')
        new_idx = f"{int(row):03d}x{int(col):03d}"
        new_idx_crd.append(new_idx)
    else:
        new_idx_crd.append(idx)       
coord.index = new_idx_crd

print(coord)

               Y       X          r
010x026   5741.5  3845.3  45.125581
010x028   6142.9  3841.4  45.125581
010x030   6549.5  3834.4  45.125581
010x032   6951.6  3839.6  45.125581
010x034   7355.5  3839.4  45.125581
...          ...     ...        ...
009x041   8741.4  3648.8  45.125581
009x043   9141.0  3650.9  45.125581
009x045   9541.6  3642.4  45.125581
009x047   9940.0  3642.1  45.125581
009x049  10337.9  3649.0  45.125581

[666 rows x 3 columns]


In [7]:
# 2. count 파일 전처리
count = pd.read_csv(count_path, index_col=0)

new_idx_cnt = []
for idx in count.index:
    parts = idx.rsplit('_', 1)
    if len(parts) == 2 and 'x' in parts[1]:
        row, col = parts[1].split('x')
        new_idx = f"{int(row):03d}x{int(col):03d}"
        new_idx_cnt.append(new_idx)
    else:
        new_idx_cnt.append(idx)       
count.index = new_idx_cnt

print(count)

         MIR1302-2  RP11-34P13.7  RP11-34P13.14  FO538757.1  RP4-669L17.10  \
010x026        0.0           0.0            0.0         2.0            0.0   
010x028        0.0           0.0            0.0         0.0            0.0   
010x030        0.0           0.0            0.0         0.0            0.0   
010x032        0.0           0.0            0.0         0.0            0.0   
010x034        0.0           0.0            0.0         0.0            0.0   
...            ...           ...            ...         ...            ...   
009x041        0.0           0.0            0.0         0.0            0.0   
009x043        0.0           0.0            0.0         0.0            0.0   
009x045        0.0           0.0            0.0         0.0            0.0   
009x047        0.0           0.0            0.0         0.0            0.0   
009x049        0.0           0.0            0.0         0.0            0.0   

         RP11-206L10.9  LINC00115  RP11-54O7.1  SAMD11  NOC2L  

In [8]:
# 3. 공통 spot merge
common_spots = count.index.intersection(coord.index)

count = count.loc[common_spots]
coord = coord.loc[common_spots, ['X', 'Y']].values

# 4. AnnData 생성
adata = sc.AnnData(count)
adata.obsm['spatial'] = coord

# 5. obs column
spatial = pd.DataFrame(
        adata.obsm['spatial'], 
        index=pd.Index(adata.obs.index, name='spot'),
        columns=['pxl_col_in_fullres', 'pxl_row_in_fullres']
)

# spatial 생성
array_rows = []
array_cols = []
for idx in spatial.index:
    try:
        row, col = str(idx).split('x')
        array_rows.append(int(row))
        array_cols.append(int(col))
    except:
        array_rows.append(0)
        array_cols.append(0)
    
spatial['array_row'] = array_rows
spatial['array_col'] = array_cols
    
# obs에 추가
adata.obs = adata.obs.join(spatial)
adata.obs['in_tissue'] = True

print(adata)

AnnData object with n_obs × n_vars = 666 × 17138
    obs: 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_row', 'array_col', 'in_tissue'
    obsm: 'spatial'


In [9]:
# 6. 이미지 처리
img = Image.open(img_path)
img_down = img.resize((max(1, img.width//10), max(1, img.height//10)))
img_array = np.array(img_down)

# uns에 추가
adata.uns['spatial'] = {
    'ST': {
        'images': {
            'downscaled_fullres': {'imgdata': img_array}
        }
    }
}

print(adata)

AnnData object with n_obs × n_vars = 666 × 17138
    obs: 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_row', 'array_col', 'in_tissue'
    uns: 'spatial'
    obsm: 'spatial'


In [10]:
# 7. 메타데이터 설정
pixel_size = 0.5 # WSI 픽셀 크기
spot_dist = 150.0 # ST는 일반적으로 150-200 픽셀

meta = {
    'pixel_size_um_estimated': pixel_size,
    'pixel_size_um_embedded': pixel_size,
    'fullres_height': img.height,
    'fullres_width': img.width,
    'spots_under_tissue': len(adata.obs),
    'spot_diameter': spot_diameter,   
    'inter_spot_dist': spot_dist                 
}

# 8. STHESTData
st_data = STHESTData(adata, img_array, pixel_size, meta)

print(st_data)

/opt/homebrew/anaconda3/envs/hest/lib/python3.9/site-packages/hestcore/wsi.py:27: UserWarning: CuImage is not available. Ensure you have a GPU and cucim installed to use GPU acceleration.
  warnings.warn("CuImage is not available. Ensure you have a GPU and cucim installed to use GPU acceleration.")


        'pixel_size' is 0.5
        'wsi' is <width=1587, height=1587, backend=NumpyWSI>
        'shapes': []


HEST data로 로드하는 데에 성공하였으므로, STimage-1K4M의 데이터를 HEST-1K와 더불어 사용 가능해졌다. 이를 통해 보다 많은 데이터셋을 사용할 수 있게 되었다. 